In [1]:
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import subprocess
import shutil
from bs4 import BeautifulSoup 
import time
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
import os
import platform
import re
from glob import glob
from urllib.request import urlopen
import requests
from tqdm import tqdm
from wordcloud import WordCloud 
# get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib import font_manager, rc
import platform
import seaborn as sns
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from collections import Counter
import IPython
import IPython.display
import missingno

In [2]:
# jupyter notebook 내에 figure를 보여주기
%matplotlib inline 

# matplotlib의 한글문제를 해결
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# font_name
rc('font', family=font_name)

## Mac OS
# rc('font', family="AppleGothic")

In [3]:
service = Service(executable_path=ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options = chrome_options)


query = "강아지사료"
p_size = str(40)
sort = "rel"

name = []
image= []
link = []
price = []
cat = []
desc_full = []
stars = []
review = []
buy = []
reg_date = []
pick = []


for p_index in range(1,251):
    request_url = f"https://search.shopping.naver.com/search/all?&pagingIndex={p_index}&pagingSize={p_size}&query={query}&sort={sort}"

    # 링크 방문
    driver.get(request_url)

    # 스크롤 끝까지 내리기

    SCROLL_PAUSE_SEC = 1

    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

#     while True:
#         # 끝까지 스크롤 다운
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#         # 1초 대기
#         time.sleep(SCROLL_PAUSE_SEC)

#         # 스크롤 다운 후 스크롤 높이 다시 가져옴
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             break
#         last_height = new_height
        
    
    # 이미지 끌어오기 위해서 천천히 스크롤 다운
    before_location = driver.execute_script("return window.pageYOffset") 
    
    while True: #현재 위치 + 500으로 스크롤 이동 
        driver.execute_script("window.scrollTo(0,{})".format(before_location + 500)) #전체 스크롤이 늘어날 때까지 대기 
        time.sleep(SCROLL_PAUSE_SEC) #이동 후 스크롤 위치 
        after_location = driver.execute_script("return window.pageYOffset") #이동후 위치와 이동 후 위치가 같으면(더 이상 스크롤이 늘어나지 않으면) 
        if before_location == after_location: 
            break #같지 않으면 다음 조건 실행 
        else: #이동여부 판단 기준이 되는 이전 위치 값 수정 
            before_location = driver.execute_script("return window.pageYOffset")

    # 제품 정보 가져오기
    for item in range(1,int(p_size)+7): #광고 6개 포함
        try:
            item_name = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[1]')
            name.append(item_name.text)
        except:
            name.append(" ")
            
        try:
            item_image = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div[1]/div[1]/div/a/img')
            image.append(item_image.get_attribute('src'))
        except:
            image.append(" ")
            
        try:
            item_link = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[1]/a')
            link.append(item_link.get_attribute('href'))
        except:
            link.append(" ")
            
        try: 
            item_price = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[2]/strong/span')
            price.append(item_price.text)
        except:
            price.append(" ")
        
        try:
            item_cat = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[3]/span[4]')
            cat.append(item_cat.text)
        except:
            cat.append(" ")
        
        try:
            item_desc = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[4]/div[1]')
            desc_full.append(item_desc.text)
        except:
            desc_full.append(" ")
            
        try:
            item_stars = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[5]/a/span')
            stars.append(item_stars.text)
        except:
            stars.append(" ")
            
        try:
            item_review = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[5]/a/em')
            review.append(item_review.text)
        except:
            review.append(" ")

        try:
            item_buy = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[5]/a[2]/em')
            buy.append(item_buy.text)
        except:
            buy.append(" ")
            
        try:
            item_reg = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[5]/span[1]')
            reg_date.append(item_reg.text)
        except:
            reg_date.append(" ")
            
        try:
            item_pick = driver.find_element_by_xpath(f'/html/body/div/div/div[2]/div[2]/div[3]/div[1]/ul/div/div[{item}]/li/div/div[2]/div[5]/span[2]/button/span/em')
            pick.append(item_pick.text)
        except:
            pick.append(" ")

# 창닫기, driver 종료
driver.close()
driver.quit()    




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Yeonjun Jeong\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [4]:
data_df = pd.DataFrame(zip(name,image,link,price,cat,desc_full,stars,review,buy,reg_date,pick),columns=['이름','이미지','링크','가격','분류','설명(전체)','평점','리뷰건수','구매건수','등록일','찜하기'])
data_df = data_df.replace(r'^\s*$',np.NaN, regex=True)
data_df = data_df.dropna(subset=['이름'])
data_df=data_df.reset_index().drop(['index'],axis=1)
data_df

,이름,이미지,링크,가격,분류,설명(전체),평점,리뷰건수,구매건수,등록일,찜하기
0,더마독 강아지 사료 관절/체중조절 2kg 골든리트리버 포메라니안 말티즈 말티푸 토이...,https://shopping-phinf.pstatic.net/main_817758...,https://adcr.naver.com/adcr?x=EhUapCGNud8jpQCs...,"29,000원",건식사료,"급여대상 : 퍼피, 어덜트, 시니어, 임신/수유, 대형견|중량 : 2kg|주원료 :...",NaN,NaN,NaN,등록일 2019.02.,129
1,더마독 강아지 사료 피부/모질/눈물 2kg 골든리트리버 포메라니안 말티즈 말티푸 토...,https://shopping-phinf.pstatic.net/main_817762...,https://adcr.naver.com/adcr?x=RgeMAXOhOXCjjixA...,"29,000원",건식사료,"급여대상 : 퍼피, 어덜트, 시니어, 임신/수유, 대형견|중량 : 2kg|주원료 :...",NaN,NaN,NaN,등록일 2019.02.,94
2,마이펫닥터 시그니처 유기농 70%함유 오가닉 피부 사료 2kg,https://shopping-phinf.pstatic.net/main_822353...,https://adcr.naver.com/adcr?x=6QQ8XYuxPyWxJ/ng...,"27,000원",건식사료,"급여대상 : 퍼피, 어덜트, 시니어, 임신/수유, 대형견|중량 : 2kg|주원료 :...",NaN,"7,366","5,149",등록일 2019.10.,481
3,마이펫닥터 시그니처 도그 푸드 슬리밍 컨트롤 다이어트 사료,https://shopping-phinf.pstatic.net/main_211407...,https://cr.shopping.naver.com/adcr.nhn?x=vtZuF...,"최저27,990원",건식사료,"급여대상 : 퍼피, 어덜트, 시니어, 임신/수유, 대형견|주원료 : 오리|등급 : ...",별점 4.8,"10,036",NaN,등록일 2019.10.,125
4,마이펫닥터 시그니처 유기농 알러제닉 눈물 사료,https://shopping-phinf.pstatic.net/main_264062...,https://cr.shopping.naver.com/adcr.nhn?x=faSRE...,"최저17,000원",건식사료,"급여대상 : 퍼피, 어덜트, 시니어, 임신/수유, 대형견|주원료 : 연어|등급 : ...",별점 4.7,"8,261",NaN,등록일 2021.03.,180
...,...,...,...,...,...,...,...,...,...,...,...
10095,스텔라앤츄이스 키블사료 치킨 터키 화이트피쉬 1.6kg,https://shopping-phinf.pstatic.net/main_832925...,https://cr.shopping.naver.com/adcr.nhn?x=lCHea...,"45,000원",건식사료,"급여대상 : 퍼피, 어덜트, 시니어, 임신/수유, 대형견|중량 : 1.6kg|주원료...",NaN,23,63,등록일 2021.07.,25
10096,뉴트리나 뉴트리나 프라임 골드 15kg 대용량 강아지사료,https://shopping-phinf.pstatic.net/main_305603...,https://cr.shopping.naver.com/adcr.nhn?x=GrWIA...,"14,990원",건식사료,NaN,NaN,1,NaN,등록일 2022.01.,0
10097,로얄캐닌 강아지사료 말티즈 어덜트 500g,https://shopping-phinf.pstatic.net/main_295363...,https://cr.shopping.naver.com/adcr.nhn?x=DM9VH...,"8,640원",건식사료,NaN,NaN,7,NaN,등록일 2021.11.,1
10098,유기농사료 오리지7 소고기 2kg,https://shopping-phinf.pstatic.net/main_822913...,https://cr.shopping.naver.com/adcr.nhn?x=4tuW3...,"26,000원",소프트사료,"급여대상 : 퍼피, 어덜트, 시니어, 임신/수유, 대형견|중량 : 2kg|주원료 :...",NaN,17,14,등록일 2022.03.,2


In [6]:
data_df.to_excel('./raw_data_df.xlsx', index = False, encoding = 'utf-8-sig')